In [10]:
import os
import time
import faker

import pandas as pd
import numpy as np

from tqdm import tqdm
import glob
import requests
from collections import Counter
from concurrent.futures import ThreadPoolExecutor

In [11]:
import re
import gcld3

In [12]:
from googletrans import Translator
from googletrans.client import Timeout

In [13]:
tqdm.pandas()

In [14]:
fake = faker.Faker(locale='en')

In [15]:
lang_detector = gcld3.NNetLanguageIdentifier(min_num_bytes=0, max_num_bytes=800)

## Clearing of all csv's, drop duplicates and full nans

In [72]:
clear_string = lambda x: re.sub(' +', ' ',
                                re.sub('https?:\/\/.+\s', ' [URL] ',
                                       re.sub('<.*?>', ' ', str(x))
                                       .replace("&nbsp;", "")
                                       .replace("&rsquo;", "'")
                                       .replace("&amp;", "&")
                                       .replace("\u202f", " ")
                                       .replace("\xa0", " ")
                                       .replace("&mdash;", "-")
                                       .replace('.', '. ')
                                       .replace('\t', ' ')
                                       .replace("•", "-")
                                       .replace("·", "-")
                                       .replace("•", "-")
                                       .replace("●", "-")
                                       .replace('\n', ' ')
                                       ).replace('/', ' / ').replace('\\', ' \ ').replace('|', ' | ')
                                ).strip()

In [73]:
csv_files = glob.glob('../data/*/*.csv')
len(csv_files)

31

In [74]:
def get_last_modified(file_paths) -> list:
    file_paths = [x[0] for x in sorted([(x, os.stat(x).st_mtime) for x in file_paths], key=lambda x: x[1], reverse=True)]
    result_list = []
    company_counter = Counter()
    for x in file_paths:
        company = x.split('/')[2]
        if company_counter[company] == 0:
            company_counter[company] += 1
            result_list.append(x)
    return result_list

In [90]:
csv_files = get_last_modified(csv_files)

In [91]:
_test = pd.read_csv(csv_files[5], index_col=0, lineterminator='\n')
_test[_test.duplicated(['company', 'title', 'internal_id'], keep=False)].sort_values('internal_id')

,title,internal_id,url,description,responsibilities,qualifications,location,category,company,publish_date


In [79]:
for file in tqdm(csv_files):
    file_name = file.rsplit('.', 1)[0]
    compnany_name = file.split('/')[2]
    df = pd.read_csv(file, index_col=0, lineterminator='\n')

    df['title'] = df['title'].apply(clear_string)
    df['description'] = df['description'].fillna('N/A').apply(clear_string)
    df['responsibilities'] = df['responsibilities'].fillna('N/A').apply(clear_string)
    df['qualifications'] = df['qualifications'].fillna('N/A').apply(clear_string)

    df.replace('N / A', None, inplace=True)
    df.drop_duplicates(subset=['company', 'title', 'internal_id'], inplace=True) # drop by company and id duplication
    df.dropna(subset=['description', 'responsibilities', 'qualifications'], how='all', inplace=True)

    df.to_csv(file_name + '.csv', line_terminator='\n')
    df.to_excel(file_name + '.xlsx', engine='xlsxwriter')

100%|██████████| 14/14 [00:20<00:00,  1.44s/it]


## Merging in one DF

In [92]:
data_frames = []
for file in tqdm(csv_files):
    file_name = file.rsplit('.', 1)[0]
    compnany_name = file.split('/')[2]

    df = pd.read_csv(file, index_col=0, lineterminator='\n')
    data_frames.append(df[['title', 'description', 'responsibilities', 'qualifications', 'company' ,'publish_date']])

merged_dataset = pd.concat(data_frames)
merged_dataset.info()

100%|██████████| 14/14 [00:01<00:00, 12.84it/s]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42019 entries, 4 to 1286
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             42019 non-null  object
 1   description       38891 non-null  object
 2   responsibilities  30850 non-null  object
 3   qualifications    40602 non-null  object
 4   company           42019 non-null  object
 5   publish_date      41165 non-null  object
dtypes: object(6)
memory usage: 2.2+ MB


In [101]:
merged_dataset['company'].value_counts()

Amazon       10000
IBM           8598
Apple         8073
Microsoft     5124
Google        3900
Tencent       1801
Sber          1286
Yandex         854
Huawei         773
MTS            678
Ozon           411
Alibaba        310
Tinkoff        164
DeepMind        47
Name: company, dtype: int64

In [104]:
nan_replaced = merged_dataset.fillna('')
merged_dataset['merged_info'] = nan_replaced['title']+ ' ; ' + nan_replaced['description'] + ' ; ' + nan_replaced['responsibilities'] + ' ; ' + nan_replaced['qualifications']

In [105]:
merged_dataset.sample(5)

,title,description,responsibilities,qualifications,company,publish_date,merged_info
500,"Program Manager II, Supply and Demand Planning",A problem isn’t truly solved until it’s solved...,Develop short and long-term supply and demand ...,Minimum qualifications: Bachelor's degree or e...,Google,2022-07-18 19:01:33.841,"Program Manager II, Supply and Demand Planning..."
2866,Application Developer: Adobe Experience Manager,"As an Application Developer, you will lead IBM...","You will translate complex functional, technic...",Minimum 4+ years in development using DevOps a...,IBM,2022-07-27 14:44:11,Application Developer: Adobe Experience Manage...
1017,Senior Software Engineer,Do you enjoy solving business problems through...,Collaborate with delivery teams and customers'...,Required Qualifications: At least 5 years of c...,Microsoft,2022-07-29 21:24:00,Senior Software Engineer ; Do you enjoy solvin...
368,"Инженер по автоматизации тестирования, Склад",Вам предстоит заняться тестированием одной из ...,NaN,NaN,Ozon,2022-07-18 10:02:24,"Инженер по автоматизации тестирования, Склад ;..."
4679,"Software Development Engineer, Fulfillment Tra...",Job summary Amazon operates one of the world's...,NaN,1+ years of experience contributing to the sys...,Amazon,2022-08-01,"Software Development Engineer, Fulfillment Tra..."


## Translating to english of merged_info

### Google API

In [106]:
def google_translate_text(text: str, from_lang='auto', g_translator=Translator(user_agent=fake.chrome(), timeout=Timeout(10)), char_limit=5000):
    try:
        text = g_translator.translate(text[:char_limit], dest='en', src=from_lang).text
    except Exception as e:
        print(e)
    time.sleep(0.2) # 200 ms wait
    return text

### LibriTranslate API

In [95]:
LIBRE_TRANSLATE_API_KEY = "8fbcf6c1-5e6d-4391-956c-c854da647f9e"
LIBRE_API_URL = "https://libretranslate.com/translate"
# LIBRE_TRANSLATE_API_KEY = ""
# LIBRE_API_URL = "http://84.38.185.155:5000/translate"

In [96]:
def libri_translate_text(text: str, from_lang='auto', timeout=20, char_limit=5000):
    payload = {
        'q': text[:char_limit],
        'source': from_lang,
        'target': 'en',
        'format': 'text',
        'api_key': LIBRE_TRANSLATE_API_KEY
    }
    libre_headers = {
        'user-agent': fake.chrome(),
    }
    try:
        result = requests.post(LIBRE_API_URL, data=payload, headers=libre_headers, timeout=timeout).json()
        # print(result)
        return result['translatedText']
    except Exception as e:
        # print(e)
        return text

### Support methods

In [108]:
def translate_df_row(row: pd.Series, nan_symbol='N / A'):
    original_lang = lang_detector.FindLanguage(row['merged_info'][:200]).language
    if original_lang != 'en':
        row['merged_info'] = google_translate_text(row['merged_info'], from_lang='auto')
        row['title'] = row['merged_info'].split(';', maxsplit=1)[0]

        # row['merged_info'] = libri_translate_text(row['merged_info'], from_lang=original_lang, timeout=35)
        # row['title'] = libri_translate_text(row['title'], from_lang=original_lang)

        # if row['description'] != nan_symbol:
        #     row['description'] = libri_translate_text(row['description'], from_lang=original_lang)
        # if row['responsibilities'] != nan_symbol:
        #     row['responsibilities'] = libri_translate_text(row['responsibilities'], from_lang=original_lang)
        # if row['qualifications'] != nan_symbol:
        #     row['qualifications'] = libri_translate_text(row['qualifications'], from_lang=original_lang)

    return row

In [109]:
def df_parallel_apply(df, func, n_cores=4, axis=1):
    df_split = np.array_split(df, n_cores)
    with ThreadPoolExecutor(max_workers=n_cores) as executor:
        futures = [executor.submit(lambda: part.apply(func, axis=axis)) for part in df_split]
        executor.shutdown(wait=True)
    transformed_parts = [f.result() for f in futures]
    return pd.concat(transformed_parts)

In [115]:
%%time
test_text = 'Первые впечатления Переночевав в гостинице в Гуаякиле, мы сели к агенту в машину и поехали на судно в Пуэрто Боливар. Доехали вопреки ожиданиям быстро, примерно за 3-4 часа. Погода была пасмурная и даже не смотря на то, что мы находимся недалеко от экватора, было прохладно. Почти все время, пока мы ехали, по обе стороны дороги были банановые плантации, но все равно в голове не укладывается: эти бананы грузят на суда в нескольких портах Эквадора десятками тысяч тонн каждый день, круглый год. Это ж несчастные бананы должны расти быстрее чем грибы. Дороги. Дороги в Эквадоре практически идеальные, хотя населенные пункты выглядят очень бедно. На дорогах много интересных машин, например очень много грузовиков - древних Фордов, которые я никогда раньше не видел. А еще несколько раз на глаза попадались старенькие Жигули :) А еще если кого-то обгоняешь и есть встречная машина, она обязательно включает фары. На больших машинах - грузовиках и автобусах, обязательно красуется местный тюнинг: машины разукрашенные, либо в наклейках, и обязательно везде огромное множество светодиодов, как будто новогодние елки едут и переливаются всеми цветами. Судно. На первый взгляд судно неплохое, в относительно хорошем состоянии, хотя и 92 года постройки. Экипаж 19 человек - 11 русских и 8 филиппинцев, включая повара. Говорят, периодически становится тоскливо от егошних кулинарных изысков. Филиппинцы здесь рядовой состав, за ними постоянно нужно следить чтобы не натворили чего, среди них только один матрос по-настоящему ответственный и с руками из нужного места, все понимает с полуслова. Остальные - типичные Равшаны да Джамшуты. А еще один из них - О___о, в добавок к этому он опасный человек, в том плане, что легко впадает в состояние ступора и отключает мозг: был случай как он закрыл одного матроса в трюме, тот орал и тарабанил внутри, это заметил боцман, начал орать на этого персонажа, который, в свою очередь испуганно выпучив глаза, трясущимися руками продолжал закручивать барашки. В итоге боцман его отодвинул и выпустил матроса из трюма. Общение на английском языке, но из-за акцента не всегда с первого раз понятно что филиппинцы говорят, особенно по рации. Напимер, говорит он тебе: Бикарпуль! Бикарпуль! А потом, когда уже поздно, выясняется что это было "Be careful!" Работа. Сразу, как только мы заселились, я не успел разложить вещи, как в мою голову ворвался такой поток информации, что ни в сказке сказать, ни топором не вырубить. Во-первых, на судне абсолютно все бумаги - мануалы, журналы, и так далее - все на английском языке. Даже блокнотик, в который записываются отчеты по грузовым операциям - и тот на английском. Бумаги... ооооо... Их тысячи, лежат в сотнях папок, плюс огромное количество документов на компьютерах. Это мне просто разорвало мозг в клочья, потому что с этим объемом информации надо ознакомиться и научиться работать в кротчайшие сроки. Постоянная беготня, постоянная суета, совсем не легко. А также надо как можно быстрее разобраться со всем оборудованием на мостике, а там его мама не горюй. В общем, пока что, свободного времени нет вообще. Абсолютно. Только ночью с 00:00 до 06:00 можно поспать. Но это продлится не долго, буквально 1-2 недели, потом океанский переход до Европы, можно будет уже спокойно стоять вахты, а в свободное время читать книги компании Seatrade, на случай если в Европе придет проверка и будет задавать вопросы. Ну и немного о приятном. Неплохая одноместная каюта. Внутри несколько шкафов и полок, удобная кровать, койка напередохнуть, стол, стул, умывальник и внутрисудовой телефон. Также выдали 2 белых офицерских комбинезона с символикой компании, каску и персональную рацию. В моем распоряжении офицерский душ со стиральной машинкой и офицерская столовая. Во время швартовых операций мое место на мостике. Хотя и матросы на палубе не сильно напрягаются - там установлены гидравлические лебедки, не надо швартовные концы тягать вручную. На этом пока все, фоток пока что нет, потому что просто некогда этим заниматься. Даже этот текст я пишу потому что просто появилось свободных 2 часа, отпустили отдохнуть, так как впереди 6-часовая вахта, а сразу после нее отшвартовка и идем в Гуаякиль, а во время отшвартовки и пока лоцман на судне - мое место на мостике. Так что, предстоит основательно заколебаться. Возле входа в канал находится якорная стоянка, где куча судов стоит и ждет своей очереди на проход. Но, нам дали добро на проход практически без ожидания и постояв на якоре всего три часа, приехал лоцман, скомандовал "dead slow ahead" (самый малый вперед), я перевел ручку машинного телеграфа, похожую на автомобильный рычаг автоматической коробки передач в указанное положение, на что телеграф отозвался громким сигналом. Стрелка тахометра поползла вверх, судно слегка задрожало и мы медленно но верно начали набирать скорость и двигаться в сторону канала. Вход в канал со стороны океана выделен буями - десятки буев, выстроенные в две линии показывают единственный верный путь, выгладит грандиозно. А справа, за пальмами и дорогими виллами со стоящими рядом с ними роскошными яхтами, простирется огромный город небоскребов - Панама. А через десять минут открылось еще одно не менее грандиозное зрелище - строительство новых шлюзов, которые будут пропускать суда бОльших размеров, чем нынешние шлюзы. Но тут впереди из-за поворота показалась огромная стена, которая двигалась в нашу сторону. Навстречу нам шло судно-автомобилевоз Green Cove и так как канал - место совсем не широкое, автомобилевоз прошел от нас всего в 50 метрах, непривычно было стоя на мостике наблюдать столь близкое расхождение. А вот и буксиры. Это значит что мы подходим к шлюзам. Только мы подошли к шлюзу, как тут же нас привязали к резвым локомотивам - Мулам. Дальше все зависело только от них. 4 Мула держали нас с разных сторон и уже они руководили процессом, они нас резво разгоняли и останавливали в нужном месте и терпеливо ждали пока шлюз не наполнится водой. Так как наше судно не очень широкое, проблем с заходом в шлюз не было, а по длине получилось так, что за нами сзади поместилось еще одно небольшое судно. Правда мы оказались более быстроходными и пока мы шли от тихоокеанских до атлантических шлюзов по каналу и озеру Гатун, это судно безнадежно отстало, но нам пришлось его ждать :('
print(test_text.split(' ').__len__(), test_text[:5000].split(' ').__len__())
google_translate_text(test_text, from_lang='auto').split(' ').__len__()

971 766
CPU times: user 17.7 ms, sys: 0 ns, total: 17.7 ms
Wall time: 327 ms


991

In [114]:
%%time
with ThreadPoolExecutor(max_workers=4) as executor:
    futures = [executor.submit(lambda: google_translate_text(test_text[:5000], from_lang='ru')) for i in range(4)]
    executor.shutdown(wait=True)
transformed_parts = [f.result() for f in futures]
[x.split(' ').__len__() for x in transformed_parts]

CPU times: user 74.9 ms, sys: 4.16 ms, total: 79 ms
Wall time: 460 ms


[991, 991, 991, 991]

### Transalting & cleaning process

In [116]:
%%time
merged_dataset[merged_dataset['company'] == 'Sber'][:5].progress_apply(translate_df_row, axis=1)

100%|██████████| 5/5 [00:21<00:00,  4.21s/it]

CPU times: user 104 ms, sys: 16.5 ms, total: 121 ms
Wall time: 21 s


,title,description,responsibilities,qualifications,company,publish_date,merged_info
0,Financial consultant (delivery of banking prod...,NaN,Финансовый консультант – это новая перспективн...,"Ты нам подходишь, если: - Ранее занимался дост...",Sber,2022-01-12 00:00:00,Financial consultant (delivery of banking prod...
1,Traineer traineer (investment),Сбер - это крупнейшая цифровая экосистема. Тех...,"Тестирование веб приложений, мобильных приложе...","Практический опыт функционального, интеграцион...",Sber,2021-12-09 00:00:00,Traineer traineer (investment); Sber is the la...
2,Network expert (Remote Access),NaN,Развитие сервиса Remote access в банке; Админи...,Высшее техническое образование; Опыт работы в ...,Sber,2022-02-22 00:00:00,Network expert (Remote Access); ; Development ...
3,QA Automation Engineer (project mlstorager),О компании: АО «Сбербанк-Технологии» - российс...,Мы ценим: Готовность работать самостоятельно и...,"Мы ищем человека, который: Имеет опыт работы т...",Sber,2022-02-28 00:00:00,QA Automation Engineer (project mlstorager); A...
4,Icassator driver,Водители-инкассаторы— это настоящие профессион...,"Тебе предстоит: перевозить денежные средства, ...",Мы ждем от тебя: готовность работать на маршру...,Sber,2022-02-28 00:00:00,Icassator driver; INCASSITA drivers are real p...


In [117]:
%%time
translated_merged_dataset = merged_dataset.progress_apply(translate_df_row, axis=1)

 98%|█████████▊| 41194/42019 [2:05:27<15:30,  1.13s/it]   

The read operation timed out


100%|██████████| 42019/42019 [2:24:15<00:00,  4.85it/s]

CPU times: user 2min 19s, sys: 7.37 s, total: 2min 27s
Wall time: 2h 24min 15s


In [118]:
translated_merged_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42019 entries, 4 to 1286
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             42019 non-null  object
 1   description       38891 non-null  object
 2   responsibilities  30850 non-null  object
 3   qualifications    40602 non-null  object
 4   company           42019 non-null  object
 5   publish_date      41165 non-null  object
 6   merged_info       42019 non-null  object
dtypes: object(7)
memory usage: 2.6+ MB


In [120]:
translated_merged_dataset['merged_info'].progress_apply(lambda x: lang_detector.FindLanguage(x[:400]).language).value_counts()

100%|██████████| 42019/42019 [00:31<00:00, 1331.52it/s]


en    41930
ja       73
fr        5
ky        3
la        2
sv        2
zh        1
sm        1
mg        1
ru        1
Name: merged_info, dtype: int64

In [124]:
translated_merged_dataset[translated_merged_dataset['company'] == 'Huawei'].sample(10)

,title,description,responsibilities,qualifications,company,publish_date,merged_info
337,Professional category] Manufacturing and adjus...,NaN,1、基于制造交付策略，制定产品制造计划，及时、高效满足产品生产交付需求。 2、负责管理供需平...,知识：熟练掌握生产计划基础知识、产品知识，供应链端到端流程。 技能：熟练掌握供需平衡管理、生...,Huawei,2022-06-06,Professional category] Manufacturing and adjus...
374,[Professional Category] Material Quality Contr...,NaN,1、依据成熟业务的流程及质量控制标准，负责成熟品类物料的试制导入、来料检测、量产质量控制及问...,知识： 1、精通物料失效分析和可靠性评估相关知识； 2、熟悉生产质量问题处理流程； 3、熟练...,Huawei,2022-06-04,[Professional Category] Material Quality Contr...
226,Assistant engineer,NaN,负责各类合作伙伴业务的日常运营工作，如Program日常运营管理、合作伙伴能力提升支持、大型...,业务技能要求： 能够参与制定多领域（行业、区域、类型）的伙伴政策架构或主导政策在本领域的有效...,Huawei,2022-06-15,Assistant engineer;; responsible for daily ope...
636,BI product manager/ demand analyst,NaN,1、 负责产品规划及设计，规划产品发展路线，打造完善的BI产品支撑分析决策; 2、 负责数据...,业务技能要求： 1、3年及以上数据类产品IT工作经历，主导或参与过企业数据分析产品的方案设计...,Huawei,2022-03-07,BI product manager/ demand analyst; 1. Respons...
676,Door lock service manager,NaN,1、 深刻洞察消费者的需求和行业趋势，制定全球领先的产品服务策略； 2、 负责终端产品服务领...,业务技能要求： 1、具体优秀的沟通推动能力、项目管理能力、自学能力； 2、有终端可服务性以及...,Huawei,2022-02-08,Door lock service manager; 1. In -depth insigh...
505,主任工程师,NaN,"- Collect forecast plan(include inbound plan, ...",业务技能要求： - Good organization skills and good te...,Huawei,2022-05-10,主任工程师 ; ; - Collect forecast plan(include inb...
616,Channel sales commissioner,NaN,参与所属领域的合作伙伴规划，在指导下负责某个领域合作伙伴拓展、合作伙伴联合营销活动，实现所负...,业务技能要求： 了解、执行业务范围内MPR、OTO等流程；具有伙伴合作能力、团队管理能力、营...,Huawei,2022-03-11,Channel sales commissioner;; partnership plann...
408,Senior fund manager (e -commerce),NaN,1、电商支付核心系统规划及建设。 2、资金通道、清结算日常运营。 3、第三方支付公司相关政策...,业务技能要求： 1、学习能力和适应性强，具有较好的分析和逻辑思维能力，良好的沟通和跨部门协作...,Huawei,2022-05-27,Senior fund manager (e -commerce); 1.; 1. E -c...
3,Assistant product engineers,NaN,1、代表处存量产品销售责任人，依托存量格局及集采份额 / 价格，对单场景 / 产品 / 区域...,业务技能要求： 掌握运营商市场分析、营销策划、项目投标和拓展等工具和方法 专业知识要求： 1...,Huawei,2022-08-09,Assistant product engineers; 1. Responsible pe...
112,Intelligent driving perception algorithm engineer,NaN,1. 针对自动驾驶感知算法业务，研发相应的深度学习算法，保持技术的先进性，用于计算机视觉，激...,业务技能要求： 1. 熟练掌握基于深度学习的图像检测，语义分割，结构化特征提取等常见任务 2...,Huawei,2022-07-07,Intelligent driving perception algorithm engin...


In [125]:
remove_non_lat = lambda x: re.sub(' +', ' ', re.sub(r'[^a-zA-Z\d\s\',.:!?()\[\]&{};-]', ' ', x)).strip()

In [126]:
translated_merged_dataset['merged_info'] = translated_merged_dataset['merged_info'].progress_apply(remove_non_lat)

100%|██████████| 42019/42019 [00:04<00:00, 8921.72it/s] 


In [127]:
translated_merged_dataset.to_csv('../results/translated_merged_dataset.csv', line_terminator='\n')
translated_merged_dataset.to_excel('../results/translated_merged_dataset.xlsx', engine='xlsxwriter')

In [132]:
translated_merged_dataset.sample(20)

,title,description,responsibilities,qualifications,company,publish_date,merged_info
7302,IBM Hybrid Cloud Modernization Architect - Wes...,"As an IBM Application Architect, you directly ...",IBM Consulting helps our clients fundamentally...,5+ years of consulting experience – Platform e...,IBM,2022-08-11 02:40:45,IBM Hybrid Cloud Modernization Architect - Wes...
218,"Software Developer Intern, Bachelor's, Winter ...",Please complete your application before August...,Create and support a productive and innovative...,Minimum qualifications: Currently pursuing a B...,Google,2022-08-02 16:31:10.110,"Software Developer Intern, Bachelor's, Winter ..."
3260,"Analyst (SQL), Retail Efficiency (RBS)",Job summary About Amazon: Work hard. Have fun....,NaN,Basic Qualifications - Proficient extracting a...,Amazon,2022-08-03,"Analyst (SQL), Retail Efficiency (RBS) ; Job s..."
4766,Practitioner - Finance and Administration Deli...,The Finance & Admin Business Analyst is essent...,"As Delivery Practitioner, you are responsible ...",Minimum 0-1 year work experience in Finance an...,IBM,2022-08-06 11:12:02,Practitioner - Finance and Administration Deli...
2787,Cellular Software Development Engineer,Join the Wireless Technology and Ecosystems te...,Apple’s Cellular Software team is seeking tale...,At least 5 years experience with cellular prot...,Apple,2022-07-04,Cellular Software Development Engineer ; Join ...
2549,Digital Solution Area Specialist _ Surface,Microsoft is on a mission to empower every per...,As a Digital Sales Surface Specialist you will...,Required Qualifications: 5-6+ years of experie...,Microsoft,2022-07-12 06:37:00,Digital Solution Area Specialist Surface ; Mic...
4173,Senior Software Engineer - M365 Network Isolation,Office 365 is at the center of Microsoft's clo...,"Design, implement, test and deliver tools and ...",5 years of software development experience in ...,Microsoft,2022-05-19 05:39:00,Senior Software Engineer - M365 Network Isolat...
1479,Package Consultant: SAP FIN FI,"As a Package Consultant at IBM, get ready to t...",You and IBM India – IBM's Purpose is to be ess...,12+ Years of SAP Functional experience special...,IBM,2022-07-18 18:44:41,Package Consultant: SAP FIN FI ; As a Package ...
7510,Penetration Testing Engineer,Job summary Do you enjoy finding unique securi...,NaN,"A Bachelor’s degree in Computer Science, Cyber...",Amazon,2022-07-25,Penetration Testing Engineer ; Job summary Do ...
59,Expert on analysis and quality control of the ...,МТС — это мультисервисная цифровая экосистема....,NaN,NaN,MTS,2022-08-09 14:16:12,Expert on analysis and quality control of the ...
